In [6]:
import pandas as pd

# Specify the path to your CSV file
file_path = r'/workspaces/group-b-news/True.csv'

# Read the CSV file into a DataFrame
df1 = pd.read_csv(file_path)

# Display the first few rows of the DataFrame
print(df1.head())


                                               title   
0  As U.S. budget fight looms, Republicans flip t...  \
1  U.S. military to accept transgender recruits o...   
2  Senior U.S. Republican senator: 'Let Mr. Muell...   
3  FBI Russia probe helped by Australian diplomat...   
4  Trump wants Postal Service to charge 'much mor...   

                                                text       subject   
0  WASHINGTON (Reuters) - The head of a conservat...  politicsNews  \
1  WASHINGTON (Reuters) - Transgender people will...  politicsNews   
2  WASHINGTON (Reuters) - The special counsel inv...  politicsNews   
3  WASHINGTON (Reuters) - Trump campaign adviser ...  politicsNews   
4  SEATTLE/WASHINGTON (Reuters) - President Donal...  politicsNews   

                 date  
0  December 31, 2017   
1  December 29, 2017   
2  December 31, 2017   
3  December 30, 2017   
4  December 29, 2017   


In [4]:
import pandas as pd

# Correctly formatted file path using raw string
file_path = r'/workspaces/group-b-news/Fake.csv'

# Read the CSV file into a DataFrame
df2 = pd.read_csv(file_path)

# Display the first few rows of the DataFrame
print(df2.head())


                                               title   
0   Donald Trump Sends Out Embarrassing New Year’...  \
1   Drunk Bragging Trump Staffer Started Russian ...   
2   Sheriff David Clarke Becomes An Internet Joke...   
3   Trump Is So Obsessed He Even Has Obama’s Name...   
4   Pope Francis Just Called Out Donald Trump Dur...   

                                                text subject   
0  Donald Trump just couldn t wish all Americans ...    News  \
1  House Intelligence Committee Chairman Devin Nu...    News   
2  On Friday, it was revealed that former Milwauk...    News   
3  On Christmas day, Donald Trump announced that ...    News   
4  Pope Francis used his annual Christmas Day mes...    News   

                date  
0  December 31, 2017  
1  December 31, 2017  
2  December 30, 2017  
3  December 29, 2017  
4  December 25, 2017  


In [17]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

# File paths
file_path1 = r'/workspaces/group-b-news/True.csv'
file_path2 = r'/workspaces/group-b-news/Fake.csv'

# Read the CSV files into DataFrames
df1 = pd.read_csv(file_path1)
df2 = pd.read_csv(file_path2)

# Extract the 'title' column from each DataFrame
headlines1 = df1['title'].tolist()
headlines2 = df2['title'].tolist()

# Combine the headlines from both DataFrames
all_headlines = headlines1 + headlines2

# Convert to DataFrame
df = pd.DataFrame({'headline': all_headlines})

# Initialize CountVectorizer
vectorizer = CountVectorizer()

# Fit and transform the headlines to get the one-hot encoding (sparse matrix)
X = vectorizer.fit_transform(df['headline'])

# Convert sparse matrix to DataFrame
one_hot_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())

# Combine the one-hot encoded data with the original DataFrame
df = pd.concat([df, one_hot_df], axis=1)

# Display the DataFrame
print(df)


                                                headline  00  000  000m  000s   
0      As U.S. budget fight looms, Republicans flip t...   0    0     0     0  \
1      U.S. military to accept transgender recruits o...   0    0     0     0   
2      Senior U.S. Republican senator: 'Let Mr. Muell...   0    0     0     0   
3      FBI Russia probe helped by Australian diplomat...   0    0     0     0   
4      Trump wants Postal Service to charge 'much mor...   0    0     0     0   
...                                                  ...  ..  ...   ...   ...   
44893  McPain: John McCain Furious That Iran Treated ...   0    0     0     0   
44894  JUSTICE? Yahoo Settles E-mail Privacy Class-ac...   0    0     0     0   
44895  Sunnistan: US and Allied ‘Safe Zone’ Plan to T...   0    0     0     0   
44896  How to Blow $700 Million: Al Jazeera America F...   0    0     0     0   
44897  10 U.S. Navy Sailors Held by Iranian Military ...   0    0     0     0   

       0045  0111  0112  01

In [2]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
import tensorflow as tf

# File paths
file_path1 = r'/workspaces/group-b-news/True.csv'
file_path2 = r'/workspaces/group-b-news/Fake.csv'

# Read the CSV files into DataFrames
df1 = pd.read_csv(file_path1)
df2 = pd.read_csv(file_path2)

# Extract the 'title' column from each DataFrame
headlines1 = df1['title'].tolist()
headlines2 = df2['title'].tolist()

# Combine the headlines from both DataFrames
all_headlines = headlines1 + headlines2

# Sample a subset of the data for testing
sample_size = 5000  # Adjust as needed
all_headlines = all_headlines[:sample_size]

# Convert to DataFrame
df = pd.DataFrame({'headline': all_headlines})

# Initialize CountVectorizer with a limit on the number of features
vectorizer = CountVectorizer(max_features=5000)  # Adjust max_features as needed

# Fit and transform the headlines to get the one-hot encoding (sparse matrix)
X = vectorizer.fit_transform(df['headline'])

# Convert sparse matrix to TensorFlow sparse tensor
def to_sparse_tensor(sparse_matrix):
    indices = np.array(list(zip(*sparse_matrix.nonzero())))
    values = sparse_matrix.data
    shape = sparse_matrix.shape
    sparse_tensor = tf.sparse.SparseTensor(indices=indices, values=values, dense_shape=shape)
    return tf.sparse.reorder(sparse_tensor)  # Reorder sparse tensor indices

sparse_tensor = to_sparse_tensor(X)

# Determine the vocabulary size and sequence length
vocab_size = len(vectorizer.get_feature_names_out())
sequence_length = X.shape[1]  # Use X.shape[1] for the number of features

# Initialize the Sequential model
model = Sequential()

# Add an Embedding layer
embedding_dim = 50  # This is the size of the dense vectors
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=sequence_length))

# Add an LSTM layer
model.add(LSTM(100, return_sequences=False))  # 100 LSTM units

# Add a Dense layer for output (binary classification example)
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print the model summary
model.summary()

# Example: Run the sparse tensor through the model
# Convert sparse tensor to dense for prediction
output = model.predict(tf.sparse.to_dense(sparse_tensor))
print("Output from the model:\n", output)


c:\Users\ADMIN\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

157/157 ━━━━━━━━━━━━━━━━━━━━ 92s 578ms/step
Output from the model:
 [[0.49965474]
 [0.49965474]
 [0.49965474]
 ...
 [0.49965474]
 [0.49965474]
 [0.49965474]]


In [3]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

# File paths
file_path1 = r'/workspaces/group-b-news/True.csv'
file_path2 = r'/workspaces/group-b-news/Fake.csv'

# Step 1: Read the CSV files into DataFrames
df1 = pd.read_csv(file_path1)
df2 = pd.read_csv(file_path2)

# Extract the 'title' column from each DataFrame
headlines1 = df1['title'].tolist()
headlines2 = df2['title'].tolist()

# Combine the headlines from both DataFrames
headlines = headlines1 + headlines2

# Create labels (1 for True, 0 for Fake) - Adjust as needed
labels = [1] * len(headlines1) + [0] * len(headlines2)

# Step 2: Tokenize the headlines
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(headlines)
sequences = tokenizer.texts_to_sequences(headlines)

# Step 3: Pad the sequences to ensure uniform length
max_sequence_length = 10
X = pad_sequences(sequences, maxlen=max_sequence_length)

# Step 4: Prepare labels
y = np.array(labels)

# Step 5: Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 6: Define the LSTM model
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=100, input_length=max_sequence_length))
model.add(LSTM(128))
model.add(Dense(1, activation='sigmoid'))

# Step 7: Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Step 8: Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

# (Optional) Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')


Epoch 1/10


c:\Users\ADMIN\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


898/898 ━━━━━━━━━━━━━━━━━━━━ 13s 11ms/step - accuracy: 0.8910 - loss: 0.2548 - val_accuracy: 0.9534 - val_loss: 0.1227
Epoch 2/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 9s 10ms/step - accuracy: 0.9730 - loss: 0.0754 - val_accuracy: 0.9574 - val_loss: 0.1215
Epoch 3/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 10s 11ms/step - accuracy: 0.9853 - loss: 0.0426 - val_accuracy: 0.9516 - val_loss: 0.1528
Epoch 4/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 10s 11ms/step - accuracy: 0.9917 - loss: 0.0246 - val_accuracy: 0.9527 - val_loss: 0.1785
Epoch 5/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 10s 11ms/step - accuracy: 0.9940 - loss: 0.0175 - val_accuracy: 0.9503 - val_loss: 0.1817
Epoch 6/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 10s 11ms/step - accuracy: 0.9961 - loss: 0.0122 - val_accuracy: 0.9524 - val_loss: 0.2261
Epoch 7/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 10s 11ms/step - accuracy: 0.9976 - loss: 0.0079 - val_accuracy: 0.9506 - val_loss: 0.2470
Epoch 8/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 10s 11ms/step - accuracy: 0.9971 - loss: 0.0082 - val_accuracy

In [4]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

# File paths
file_path1 = r'/workspaces/group-b-news/True.csv'
file_path2 = r'/workspaces/group-b-news/Fake.csv'

# Read the CSV files into DataFrames
df1 = pd.read_csv(file_path1)
df2 = pd.read_csv(file_path2)

# Extract the 'title' column from each DataFrame
headlines1 = df1['title'].tolist()
headlines2 = df2['title'].tolist()

# Combine the headlines from both DataFrames
headlines = headlines1 + headlines2

# Create labels (1 for True, 0 for Fake)
labels = [1] * len(headlines1) + [0] * len(headlines2)

# Tokenize the headlines
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(headlines)
sequences = tokenizer.texts_to_sequences(headlines)

# Pad the sequences to ensure uniform length
max_sequence_length = 10
X = pad_sequences(sequences, maxlen=max_sequence_length)

# Prepare labels
y = np.array(labels)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the LSTM model
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=100, input_length=max_sequence_length))
model.add(LSTM(128))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Predict on the entire dataset
predictions = model.predict(X)

# Convert predictions to binary (0 or 1)
predicted_labels = (predictions > 0.5).astype(int).flatten()

# Create a DataFrame to store results
results_df = pd.DataFrame({
    'headline': headlines,
    'actual_label': labels,
    'predicted_label': predicted_labels,
    'prediction_text': ['This news is real' if label == 1 else 'This news is fake' for label in predicted_labels]
})

# Print results
print(results_df[['headline', 'prediction_text']])


Epoch 1/10


c:\Users\ADMIN\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


898/898 ━━━━━━━━━━━━━━━━━━━━ 13s 11ms/step - accuracy: 0.8994 - loss: 0.2481 - val_accuracy: 0.9516 - val_loss: 0.1280
Epoch 2/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 9s 10ms/step - accuracy: 0.9738 - loss: 0.0733 - val_accuracy: 0.9552 - val_loss: 0.1286
Epoch 3/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 9s 10ms/step - accuracy: 0.9873 - loss: 0.0393 - val_accuracy: 0.9542 - val_loss: 0.1576
Epoch 4/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 9s 10ms/step - accuracy: 0.9911 - loss: 0.0255 - val_accuracy: 0.9531 - val_loss: 0.1822
Epoch 5/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 9s 10ms/step - accuracy: 0.9943 - loss: 0.0161 - val_accuracy: 0.9486 - val_loss: 0.2190
Epoch 6/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 9s 10ms/step - accuracy: 0.9960 - loss: 0.0123 - val_accuracy: 0.9511 - val_loss: 0.2398
Epoch 7/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 9s 10ms/step - accuracy: 0.9989 - loss: 0.0053 - val_accuracy: 0.9509 - val_loss: 0.2788
Epoch 8/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 9s 10ms/step - accuracy: 0.9984 - loss: 0.0059 - val_accuracy: 0.95